# DM-like blind rotation

For details, read follow papers

* <a href="https://eprint.iacr.org/2014/816"> FHEW: Bootstrapping Homomorphic Encryption in less than a second </a>
* <a href="https://eprint.iacr.org/2020/086"> Bootstrapping in FHEW-like Cryptosystems </a>

---

In [1]:
import numpy as np
from note_include.elem.Ring import Ring
from note_include.FHEW      import FHEW
from note_include.elem.RLWE import RLWE

np.set_printoptions(threshold=np.inf, linewidth=np.inf)

---

In [2]:
q    = 128
n    = 16
Q    = 128 # For N = 1024
N    = 64  
B    = 256
B_ks = 256
std  = 0

We should define message in binary domain, $m \in \{0,1\}$.   
We can define encryption method as follows:

>$\textbf{Definition}$
>
>Let $m \in \{0,1\}$ is a message, $\vec{a}$ is a vector of random number $a_i \in \Z_q$, $\vec{s}$ is a secret values, and $e$ is a small noise value.   
>Then we can define encryption method as follows:
>
>\begin{align*}
>    \textsf{FHEW.enc}(m; \vec{s}) = (\vec{a}, b), \text{ where } b = \lang \vec{a}, \vec{s} \rang + m \cdot q/4 + e.
>\end{align*}
>
>The decryption method can be easily defined as follows:
>
>\begin{align*}
>    \textsf{FHEW.dec}((a,b); \vec{s}) = b - \lang \vec{a}, \vec{s} \rang = m \cdot q/4 + e \approx m \cdot q/4.
>\end{align*}

In [3]:
fhew = FHEW(n, q, N, Q, std, B, B_ks)
rlwe = RLWE(N, Q, std)
s, s_ring = fhew.dm_keygen()


In [4]:
# s_count = 0
# num = 20
# for i in range(num):
#     ct0 = fhew.encrypt(0, s)
#     ct1 = fhew.encrypt(1, s)
#     ct_and = fhew.evalBin(ct0, ct1, "AND")

#     ptxt_1and0 = fhew.decrypt(ct_and, s)
#     if(ptxt_1and0 == 0):
#         s_count += 1

# print("Success Count : ", s_count, "/", num)
#     # print(ptxt_1and0, ", ---It should be 0 ( 1 AND 0 = 0 ) ---")

In [5]:
def nega_ACC_init(b, q, N):
    acc = np.zeros(N)
    for i in range(N//2):
        acc[i]             =  (b-(i*4)) % q
        acc[i + (N//2)]    = -(b-(i*4)) % q
    acc = Ring(N, q, acc)
    return (Ring(N, q, np.zeros(N)), acc)

In [ ]:
ct0 = fhew.encrypt(0, s)
ct1 = fhew.encrypt(1, s)

ctxt_operand =  fhew.LWE_CC.add(ct0, ct1)
a, b = ctxt_operand
print((b - np.dot(a, s)) % q)

# acc = fhew.acc_init(ctxt_operand, "AND")
acc = nega_ACC_init(b, q, N)

operanded_a, operanded_b = ctxt_operand
acc_a, acc_b = acc

tmp_acc     = acc
_, test_acc = acc
print("Ideal extraction : ", test_acc[int((b - np.dot(a, s)) % q)])
print("ACC : ", test_acc)

# Blind rotation
print(operanded_a)

# [Test] set encryption with rlwe
tmp_acc = fhew.RLWE_CC.encrypt(test_acc, s_ring)

for i, _a in enumerate(operanded_a):
    # print(i, " : ", "as : ", _a * s[i] % fhew.q)
    if(_a == 0):continue
    for j in range(fhew.d_g):
        v = _a % fhew.B
        _a = _a // fhew.B
        # if v == 0 : continue
        tmp_acc = fhew.RGSW_CC.mult_rlwe(tmp_acc, fhew.brk[i][v][j])

    acc_decrypted = fhew.RLWE_CC.decrypt(tmp_acc, s_ring)
    print(acc_decrypted)

print(acc_decrypted.coeffs[0])

32.0
Ideal extraction :  44.0
[  6  95  73  38 115 105  11 112 117  84  53  22  51  98 115 117]
R(n=64, q=128, coeffs= 84 + 80x + 76x^2 + 72x^3 + 68x^4 + 64x^5 + 60x^6 + 56x^7 + 52x^8 + 48x^9 + 44x^10 + 40x^11 + 36x^12 + 32x^13 + 28x^14 + 24x^15 + 20x^16 + 16x^17 + 12x^18 + 8x^19 + 4x^20 + 0x^21 + 124x^22 + 120x^23 + 116x^24 + 112x^25 + 108x^26 + 104x^27 + 100x^28 + 96x^29 + 92x^30 + 88x^31 + 44x^32 + 48x^33 + 52x^34 + 56x^35 + 60x^36 + 64x^37 + 68x^38 + 72x^39 + 76x^40 + 80x^41 + 84x^42 + 88x^43 + 92x^44 + 96x^45 + 100x^46 + 104x^47 + 108x^48 + 112x^49 + 116x^50 + 120x^51 + 124x^52 + 0x^53 + 4x^54 + 8x^55 + 12x^56 + 16x^57 + 20x^58 + 24x^59 + 28x^60 + 32x^61 + 36x^62 + 40x^63)
R(n=64, q=128, coeffs= 116 + 112x + 108x^2 + 104x^3 + 100x^4 + 96x^5 + 92x^6 + 88x^7 + 84x^8 + 80x^9 + 76x^10 + 72x^11 + 68x^12 + 64x^13 + 60x^14 + 56x^15 + 52x^16 + 48x^17 + 44x^18 + 40x^19 + 36x^20 + 32x^21 + 28x^22 + 24x^23 + 20x^24 + 16x^25 + 12x^26 + 8x^27 + 4x^28 + 0x^29 + 124x^30 + 120x^31 + 116x^32 + 112

In [7]:
aaa = np.array(operanded_a)
sss = np.array(s)

print(q - ((aaa * sss) % q))

[128.  88.  31.  72.  72. 109. 108. 128.  35. 128. 116.  66.  92. 127.  46.  89.]
